In [1]:
import sys

!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

 This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/Chromium#Hardware_video_acceleration for more information on VAAPI video decoding support.

=== Widevine Support ===

The packages in this PPA have support for Widevine inside Chromium enabled. However, you still need to copy some files from Chrome into Chromium for you to use Netflix (or other websites using Encrypted Media Extensions) in Chromium.

1. Download and install Chrome (or extract the necessary files, if

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

webdriver_service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service = webdriver_service, options = options)

In [23]:
from selenium.webdriver.common.keys import Keys
import csv
from google.colab import drive
import numpy as np
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException, UnexpectedAlertPresentException

# CSV 파일 경로
csv_file_path = "/content/drive/MyDrive/rawdata_50.csv"

first_value = 14
second_value = 4

# 특정 행까지 읽기
target_row = 5  # 읽고자 하는 행 번호
current_row = 0  # 현재 행의 인덱스

# 2차원 배열 형태 지정
rows = target_row + 1  # 행 개수
cols = 2  # 열 개수

# 모든 행을 저장할 리스트
all_rows = []  

# 비어있는 2차원 문자열 배열 선언
array_2d = np.full((rows, cols), None, dtype=object)

# CSV 파일 열기
with open(csv_file_path, "r", newline="", encoding="CP949") as file:

    # CSV 파일 읽기
    csv_reader = csv.reader(file)
    all_rows = list(csv_reader)


start = "/html/body/div[5]/div[2]/div[2]/div[1]/div/div[1]/div[2]/form/input[1]"
arrive = "/html/body/div[5]/div[2]/div[2]/div[1]/div/div[2]/div[2]/form/input[1]"
walkButton = "/html/body/div[5]/div[2]/div[2]/div[2]/div/a[3]"
detailButton = "/html/body/div[5]/div[2]/div[2]/div[5]/div[3]/div[1]/ul/li[1]/div[1]/a";
keyWord = "횡단보도"
aptNm = ""
pumYn = 0

target_array = ["어린이집", "유치원", "초등학교", "중학교", "고등학교", "마트", "병원", "버스정류장", "지하철역"]
target_yn_arry = ["어린이집", "유치원", "초등학교"]

# 접속
url = "https://map.kakao.com/"
driver.get(url)
time.sleep(3)

head = 0

for row in all_rows:

    head += 1
    if head < 3:
      continue

    first_val = row[14]
    second_val = row[4]

    if second_val.endswith("아파트"):
      aptNm = first_val + " " + second_val
    else:
      aptNm = first_val + " " + second_val + " 아파트"

    print("########## Start : " + aptNm)

    for target in target_array:

      print("#### Target : " + target)
    
      pumYn = "0"

      # 길찾기 버튼 클릭
      driver.find_element(By.XPATH, "/html/body/div[5]/div[1]/div/div/ul/li[2]/a").click()
      time.sleep(3)
      
      try:
        # 1-1. 검색창 클릭 - 아파트 입력
        driver.find_element(By.XPATH, start).click()
        driver.find_element(By.XPATH, start).send_keys(aptNm)

        # 1-2. Enter눌러 검색
        driver.find_element(By.XPATH, start).send_keys(Keys.ENTER)    
        time.sleep(3)

        # 2-1. 검색창 클릭 - 주변 어린이집 입력
        driver.find_element(By.XPATH, arrive).click()     
        driver.find_element(By.XPATH, arrive).send_keys(aptNm + " 주변 " + target)    

        # 2-2. Enter눌러 검색
        driver.find_element(By.XPATH, arrive).send_keys(Keys.ENTER)     
        time.sleep(3)

        # 3. 도보버튼 클릭
        driver.find_element(By.XPATH, walkButton).click()     
        time.sleep(3) # 검색input

        # 4. meter 조회
        meter = driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div[2]/div[5]/div[3]/div[1]/ul/li[1]/div[1]/div/p/span[2]").text

        print(aptNm + " / " + target + " meter : " + meter)

        if target in target_yn_arry:
            # 5. 세부경로 조회
            driver.find_element(By.XPATH, detailButton).click()     
            time.sleep(1)

            routeText = driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div[2]/div[5]/div[3]/div[1]/ul/li[1]/div[2]/ul").text

            if keyWord not in routeText:
              pumYn = "1"

            print(aptNm + " / " + target + " pumYn : " + pumYn)

      except ElementNotInteractableException:
          print("ElementNotInteractableException : " + aptNm)
          meter = "-1"
          pumYn = "-1"

      except NoSuchElementException:
          print("NoSuchElementException : " + aptNm)
          meter = "-1"
          pumYn = "-1"

      except UnexpectedAlertPresentException:
          print("UnexpectedAlertPresentException : " + aptNm)
          meter = "-1"
          pumYn = "-1"

          url = "https://map.kakao.com/"
          driver.get(url)
          time.sleep(3)

      if target == "어린이집":
        row[15] = meter
        row[18] = pumYn
      elif target == "유치원":
        row[16] = meter
        row[19] = pumYn
      elif target == "초등학교":
        row[17] = meter
        row[20] = pumYn
      elif target == "중학교":
        row[21] = meter
      elif target == "고등학교":
        row[22] = meter
      elif target == "마트":
        row[23] = meter
      elif target == "병원":
        row[24] = meter
      elif target == "버스정류장":
        row[25] = meter
      elif target == "지하철역":
        row[26] = meter

# CSV 파일 저장
with open(csv_file_path, "w", newline="", encoding="CP949") as file:
    writer = csv.writer(file)
    writer.writerows(all_rows)
      




      # else:
      #     print(aptNm + " " + timeText + " " + str(pumYn))
      # 15 : 어린이집까지의거리
      # 16유치원, 17초등학교
      # 18 19 20 얼품아 유품아 초품아
      # 21 22 23 24 25 26 중고마병버지








########## Start : 서울 강남구 개포래미안포레스트 아파트
#### Target : 어린이집
서울 강남구 개포래미안포레스트 아파트 / 어린이집 meter : 312m
서울 강남구 개포래미안포레스트 아파트 / 어린이집 pumYn : 1
#### Target : 유치원
서울 강남구 개포래미안포레스트 아파트 / 유치원 meter : 1.0km
서울 강남구 개포래미안포레스트 아파트 / 유치원 pumYn : 0
#### Target : 초등학교
서울 강남구 개포래미안포레스트 아파트 / 초등학교 meter : 803m
서울 강남구 개포래미안포레스트 아파트 / 초등학교 pumYn : 0
#### Target : 중학교
서울 강남구 개포래미안포레스트 아파트 / 중학교 meter : 1.8km
#### Target : 고등학교
서울 강남구 개포래미안포레스트 아파트 / 고등학교 meter : 807m
#### Target : 마트
서울 강남구 개포래미안포레스트 아파트 / 마트 meter : 3.3km
#### Target : 병원
서울 강남구 개포래미안포레스트 아파트 / 병원 meter : 1.2km
#### Target : 버스정류장
서울 강남구 개포래미안포레스트 아파트 / 버스정류장 meter : 1.5km
#### Target : 지하철역
서울 강남구 개포래미안포레스트 아파트 / 지하철역 meter : 1.8km
